# LEGO Super Mario
This notebook will take a look at the GV/CVR for LSM

In [1]:
import pandas as pd

In [1]:
import gc
import os
import numpy as np
import pandas as pd
from itertools import islice
from openpyxl import load_workbook
from datetime import date, timedelta, datetime

ORDERED_REVENUE_STRING = "Ordered Revenue"
DAILY_PATH_SUFFIX = "_daily"
REPLACE_WITH_NOTHING = ""
XSLX = ".xlsx"
INVENTORY_HEALTH_STRING = "Inventory Health"
SHIPPED_COGS_STRING = "Shipped COGS"
ORDERED_REVENUE_STRING = "Ordered Revenue"
GB_ABBREVIATION = "GB"
UK_ABBREVIATION = "UK"
SPLIT_BY = "\\"
TO_REPLACE = "_daily"
REPLACE_WITH = ""
COUNTRY_COLUMN = "Country"
DATE_COLUMN = "Date"
ASIN_COLUMN = "ASIN"
ID_COLUMN = "ID"
COLUMNS_TO_DROP = ['Ordered Units', 'Ordered Units - % of Total', 'Ordered Units - Prior Period', 'Ordered Units - Last Year', 'Glance Views', 'Change in Glance View - Prior Period', 'Change in GV Last Year', 'Conversion Rate', 'Rep OOS', 'Rep OOS - % of Total', 'Rep OOS - Prior Period', 'LBB (Price)']
COLUMNS_TO_ADD_PREFIX = ["Subcategory (Sales Rank)", 'Average Sales Price', 'Average Sales Price - Prior Period']
COLUMNS_WITH_PREFIX = ["S.COGS " + column_name for column_name in COLUMNS_TO_ADD_PREFIX]
PRODUCT_TITLE_COLUMN = "Product Title"
COUNTRY_LIST = ["DE", "ES", "FR", "GB", "IT", "NL"]
ENCODING = "utf-8-sig"

def make_list_of_child_directories(root_directory):
    """
    Function uses os.path to find and return a list of the child directories inside a specified root/parent directory 

    :param root_directory: directory/location of root/parent that is to be crawled
    :returns child_directory_list: list of suffix names of child directories
    """
    root = root_directory
    child_directory_list = [item for item in os.listdir(root) if os.path.isdir(os.path.join(root, item))]
    return child_directory_list


def make_filtered_date_weekly_list(root_directory_list, path_filter):
    """
    Function filters a list of directories based on a specified string filter

    :param root_directory_list: list of suffix names of directories
    :param path_filter: string to filter list by
    :returns fitlered_list: filtered list of suffix names of directories
    """
    filtered_list = [weekly_file for weekly_file in root_directory_list if path_filter in weekly_file]
    return filtered_list


def make_directory_date_list(filtered_list, to_replace, replace_with):
    """
    Function creates a list by removing a string from each element. It sorts the list from oldest date to newest

    :param filtered_list: list of suffix names of directories
    :param to_replace: string to replace 
    :param replace_with: string to replace with
    :returns file_name_list: list of weekly folder dates
    """
    file_name_list = [weekly_file.replace(to_replace, replace_with) for weekly_file in filtered_list]
    return sorted(file_name_list)


def make_latest_available_daily_directory(file_name_list, suffix):
    """
    Function makes the path to the directory containing the latest available weekly ABA data

    :param file_name_list: list containing the weekly folder dates
    :returns latest_available_weekly_directory: path to the directory containing the latest available weekly ABA data
    """
    latest_available_daily_directory = DAILY_PATH_PLACEHOLDER.format(file_name_list[-1] + suffix)
    return latest_available_daily_directory


def make_list_of_files_in_directory(directory_path, file_extension):
    """
    Function crawls a specified directory and returns a list of files in that directory which are of specified file extension type

    :param directory_path: directory/location of root/parent that is to be crawled
    :param file_extension: file extension type
    :returns fitlered_list: list of files in that directory which are of specified file extension type
    """
    file_list = list()
    for root, dirs, files in os.walk(directory_path):
        for file in files:
            if file.endswith(file_extension):
                file_list.append(os.path.join(root, file))
    return file_list


def openpyxl_workbook_to_pandas_df(file_path):
    """
    Function makes a Pandas Dataframe from an Openpyxl workbook. This is done because there are troubles reading SharePoint files directly with Pandas
    """
    wb = load_workbook(filename=file_path)
    ws = wb.active
    data = ws.values
    cols = next(data)[0:]
    data = list(data)
    data = (islice(r, 0, None) for r in data)
    df = pd.DataFrame(data, columns=cols)
    new_header = df.iloc[0]
    df = df[1:]
    df.columns = new_header
    return df


def make_country_from_file(file_path):
    """
    Function finds the country code for a specified file using the file path

    :param file_path: directory/location of file
    :returns country_abbreviation: country code for the specified file
    """
    # if INVENTORY_HEALTH_STRING in file_path:
    #     start_slice = -7
    #     end_slice = -5
    if ORDERED_REVENUE_STRING in file_path:
        start_slice = -23
        end_slice = -21
    # if SHIPPED_COGS_STRING in file_path:
    #     start_slice = -20
    #     end_slice = -18
    country_abbreviation = file_path[start_slice:end_slice]
    if country_abbreviation == "GB":
        country_abbreviation = "UK"
    else:
        pass
    return str(country_abbreviation)


def make_date_from_file(file_path):
    """
    Function finds the date for a specified file using the file path

    :param file_path: directory/location of file
    :returns date: date for the specified file
    """
    date_daily_string = file_path.split(SPLIT_BY)[-2]
    date_string = date_daily_string.replace(TO_REPLACE, REPLACE_WITH)
    day = date_string[-2:]
    month = date_string[-4:-2]
    year = date_string[:4]
    date = day + "/" + month + "/" + year
    return date


def add_country_date_to_dataframe(dataframe, country, date):
    """
    Function adds a Country and Date column to a dataframe
    """
    dataframe[COUNTRY_COLUMN] = country
    dataframe[DATE_COLUMN] = date
    return dataframe


def make_id_column(dataframe):
    """
    Function makes an ID column for a dataframe based off the 
    """
    dataframe[ID_COLUMN] = dataframe[ASIN_COLUMN] + "_" + dataframe[COUNTRY_COLUMN] + "_" + dataframe[DATE_COLUMN]
    return dataframe


def load_clean_and_transform_single_file_pipeline(file_path):
    """
    Function serves as a pipeline for the process that is needed to load and create a country and date column into the dataframe
    """
    dataframe = openpyxl_workbook_to_pandas_df(file_path)
    country_abbreviation = make_country_from_file(file_path)
    date = make_date_from_file(file_path)
    transformed_dataframe = add_country_date_to_dataframe(dataframe, country_abbreviation, date)
    transformed_dataframe = make_id_column(transformed_dataframe)
    try:
        transformed_dataframe.rename(columns={'Glance views': 'Glance Views', 'Conversion rate': 'Conversion Rate', "Sub-category (Sales Rank)": "Subcategory (Sales Rank)"}, inplace=True)
    except:
        pass
    return transformed_dataframe

In [2]:
child_direct_list = make_list_of_child_directories(r"C:\Users\gbjaudom\LEGO\WE eComm CoE Analytics Team - Documents\Data\VC Files\Daily Files\2020")
filtered_list = make_filtered_date_weekly_list(child_direct_list, "_daily")
directory_date_list = make_directory_date_list(filtered_list, "_daily", "")
directory_date_list = directory_date_list[92:]

In [43]:
# direct_list = make_list_of_files_in_directory(r"C:\Users\gbjaudom\LEGO\WE eComm CoE Analytics Team - Documents\Data\VC Files\Daily Files\2020\20200801_daily", ".xlsx")
# direct_list_filtered_OR_files = [x for x in direct_list if 'Ordered Revenue' in x]

# empty_df = pd.DataFrame()

# for file_path in direct_list_filtered_OR_files:
#     day_country_file = load_clean_and_transform_single_file_pipeline('C:\\Users\\gbjaudom\\LEGO\\WE eComm CoE Analytics Team - Documents\\Data\\VC Files\\Daily Files\\2020\\20200801_daily\\20200801_daily_Sales Diagnostic_Detail View_DE_Ordered Revenue.xlsx')
#     empty_df = empty_df.append(day_country_file)

c:\Users\gbjaudom\OneDrive - LEGO\Documents\python_projects\miscellaneous\venv_miscellaneous\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [7]:
master_df = pd.DataFrame()

for directory in directory_date_list: # Loop through all days
    print(directory)
    direct_list = make_list_of_files_in_directory(r"C:\Users\gbjaudom\LEGO\WE eComm CoE Analytics Team - Documents\Data\VC Files\Daily Files\2020\{}_daily".format(directory), ".xlsx")
    direct_list_filtered_OR_files = [x for x in direct_list if 'Ordered Revenue' in x]
    
    empty_df = pd.DataFrame()
    for file_path in direct_list_filtered_OR_files: # Loop through OR file in day
        day_country_file = load_clean_and_transform_single_file_pipeline(file_path)
        day_country_file = day_country_file[['ASIN', 'Product Title', 'Ordered Revenue', 'Ordered Units', 'Average Sales Price', 'Glance Views', 'Conversion Rate', 'Country', 'Date', 'ID']]
        empty_df = empty_df.append(day_country_file)
        
    master_df = master_df.append(empty_df)

20200801


c:\Users\gbjaudom\OneDrive - LEGO\Documents\python_projects\miscellaneous\venv_miscellaneous\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


20200802
20200803
20200804
20200805
20200806
20200807
20200808
20200809
20200810
20200811
20200812
20200813
20200814
20200815
20200816
20200817
20200818
20200819
20200820
20200821
20200822
20200823
20200824
20200825
20200826
20200827
20200828
20200829
20200830
20200831
20200901
20200902
20200903
20200904
20200905
20200906
20200907
20200908
20200909
20200910
20200911
20200912
20200913
20200914
20200915
20200916
20200917
20200918
20200919
20200920
20200921
20200922
20200923
20200924
20200925
20200926
20200927
20200928
20200929
20200930
202010
20201001
20201002
20201003
20201004
20201005
20201006
20201007
20201008
20201009
20201010
20201011
20201012
20201013
20201014
20201015
20201016
20201017
20201018
20201019
20201020
20201021
20201022
20201023
20201024
20201025
20201026
20201027
20201028
20201029
20201030
20201031
202011
20201101
20201102
20201103
20201104
20201105
20201106
20201107
20201108
20201109
20201110
20201111
20201112
20201113
20201114
20201115
20201116
20201117
20201118
20201

In [9]:
master_df.to_csv(r"C:\Users\gbjaudom\Onedrive - LEGO\Documents\python_projects\miscellaneous\data\processed\2020_aug_dec_daily_gv_cvr_or.csv")

In [10]:
master_df

,ASIN,Product Title,Ordered Revenue,Ordered Units,Average Sales Price,Glance Views,Conversion Rate,Country,Date,ID
1,B082WDQHZQ,LEGO 71360 Super Mario Abenteuer mit Mario – S...,37998.37,938.0,40.509989,29410.0,0.031894,DE,01/08/2020,B082WDQHZQ_DE_01/08/2020
2,B0813R6JC6,LEGO 75280 Star Wars Clone Troopers der 501. L...,19059.97,789.0,24.157123,3603.0,0.218984,DE,01/08/2020,B0813R6JC6_DE_01/08/2020
3,B081P5P1SM,LEGO 75283 Star Wars Armored Assault Tank (AAT...,12936.0,385.0,33.6,3251.0,0.118425,DE,01/08/2020,B081P5P1SM_DE_01/08/2020
4,B082WF8MVV,LEGO 71369 Super Mario Bowsers Festung – Erwei...,9123.23,109.0,83.699358,6049.0,0.01802,DE,01/08/2020,B082WF8MVV_DE_01/08/2020
5,B0813QBV6V,LEGO 75969 Harry Potter Astronomieturm auf Sch...,9003.31,153.0,58.845163,2857.0,0.053553,DE,01/08/2020,B0813QBV6V_DE_01/08/2020
...,...,...,...,...,...,...,...,...,...,...
159,B082WD5BTT,LEGO 71363 Super Mario Pokey i öknen – Expansi...,-156.0,-1.0,156.0,4.0,-0.25,SE,31/12/2020,B082WD5BTT_SE_31/12/2020
160,B082WDSG3W,LEGO 71365 Super Mario Piranha Plant Power Sli...,-174.12,-1.0,174.12,0.0,—,SE,31/12/2020,B082WDSG3W_SE_31/12/2020
161,B0813RJRYG,"LEGO 41428 Friends Strandhus, Byggsats med Min...",-364.33,-1.0,364.33,3.0,-0.333333,SE,31/12/2020,B0813RJRYG_SE_31/12/2020
162,B07W5PX1QP,LEGO 41395 Friends Vänskapsbuss Byggsats med M...,-470.86,-1.0,470.86,0.0,—,SE,31/12/2020,B07W5PX1QP_SE_31/12/2020


## Merge with Internal

In [12]:
lookup = pd.read_excel(r"C:\Users\gbjaudom\LEGO\WE eComm CoE Analytics Team - Documents\Data\Reference docs\ASIN Lookup full.xlsx", sheet_name="Sheet1")
lookup.rename(columns={'Material':'material', "Comms":"communication_number", "SIOC flag":"sioc_flag", "Title":"name", "Theme": "top_theme", "Super Segment(Curr)": "super_segment", "RRP Lead EUR":"", "RRP Lead Range EUR": "rrp_lead_range_eur", "Price Range": "price_range"}, inplace=True)

In [23]:
data = master_df.merge(lookup, how="left")
data.to_csv(r"C:\Users\gbjaudom\Onedrive - LEGO\Documents\python_projects\miscellaneous\data\processed\2020_aug_dec_daily_gv_cvr_or.csv")

MemoryError: Unable to allocate 17.5 MiB for an array with shape (2, 1144194) and data type float64

## LSM Filter

In [19]:
lsm = data[data["top_theme"] == "SUPER MARIO"]
lsm

,ASIN,Product Title,Ordered Revenue,Ordered Units,Average Sales Price,Glance Views,Conversion Rate,Country,Date,ID,material,communication_number,sioc_flag,name,top_theme,super_segment,,rrp_lead_range_eur,price_range
0,B082WDQHZQ,LEGO 71360 Super Mario Abenteuer mit Mario – S...,37998.37,938.0,40.509989,29410.0,0.031894,DE,01/08/2020,B082WDQHZQ_DE_01/08/2020,6288909.0,71360.0,0.0,Adventures with Mari,SUPER MARIO,Boys,59.99,<80 EUR,MPP
3,B082WF8MVV,LEGO 71369 Super Mario Bowsers Festung – Erwei...,9123.23,109.0,83.699358,6049.0,0.01802,DE,01/08/2020,B082WF8MVV_DE_01/08/2020,6288927.0,71369.0,0.0,Bowser's Castle Boss,SUPER MARIO,Boys,99.99,<150 EUR,HPP
12,B082WF8MB5,LEGO 71367 Super Mario Marios Haus und Yoshi –...,5348.33,216.0,24.760787,4500.0,0.048,DE,01/08/2020,B082WF8MB5_DE_01/08/2020,6288923.0,71367.0,0.0,Mario's House & Yosh,SUPER MARIO,Boys,29.99,<50 EUR,LPP
19,B082WF7Z4Q,LEGO 71362 Super Mario Bewachte Festung – Erwe...,3360.8,80.0,42.01,5529.0,0.014469,DE,01/08/2020,B082WF7Z4Q_DE_01/08/2020,6288913.0,71362.0,0.0,Guarded Fortress Exp,SUPER MARIO,Boys,49.99,<50 EUR,LPP
30,B082WDZM4T,LEGO 71368 Super Mario Toads Schatzsuche – Erw...,2661.91,40.0,66.54775,2233.0,0.017913,DE,01/08/2020,B082WDZM4T_DE_01/08/2020,6288925.0,71368.0,0.0,Toad's Treasure Hunt,SUPER MARIO,Boys,79.99,<80 EUR,MPP
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1144071,B082WDQHZQ,LEGO 71360 Super Mario Äventyr med Mario – Sta...,0.0,0.0,—,4.0,0.0,SE,31/12/2020,B082WDQHZQ_SE_31/12/2020,6288909.0,71360.0,0.0,Adventures with Mari,SUPER MARIO,Boys,59.99,<80 EUR,MPP
1144072,B082WDBGMP,LEGO 71366 Super Mario Boomer Bills Attack - E...,0.0,0.0,—,0.0,—,SE,31/12/2020,B082WDBGMP_SE_31/12/2020,6288921.0,71366.0,0.0,Boomer Bill Barrage,SUPER MARIO,Boys,29.99,<50 EUR,LPP
1144073,B082WD5BV6,LEGO 71372 Super Mario Cat Mario – Boostpaket ...,0.0,0.0,—,0.0,—,SE,31/12/2020,B082WD5BV6_SE_31/12/2020,6288933.0,71372.0,0.0,Cat Mario Power-Up P,SUPER MARIO,Boys,9.99,<10 EUR,LPP
1144189,B082WD5BTT,LEGO 71363 Super Mario Pokey i öknen – Expansi...,-156.0,-1.0,156.0,4.0,-0.25,SE,31/12/2020,B082WD5BTT_SE_31/12/2020,6288915.0,71363.0,0.0,Desert Pokey Expansi,SUPER MARIO,Boys,19.99,<20 EUR,LPP


### Country

In [1]:
import pandas as pd

In [5]:
uk = pd.read_csv(r"C:\Users\gbjaudom\LEGO\WE eComm CoE Analytics Team - Documents\Data\daily_vc_data\uk_2021_master_aba_vc_data.csv")
uk = uk[['ASIN', 'Product Title', 'Ordered Revenue', 'Ordered Units', 'Average Sales Price', 'Glance Views', 'Conversion Rate', 'Country', 'Date', 'ID']]
lookup = pd.read_excel(r"C:\Users\gbjaudom\LEGO\WE eComm CoE Analytics Team - Documents\Data\Reference docs\ASIN Lookup full.xlsx", sheet_name="Sheet1")
lookup.rename(columns={'Material':'material', "Comms":"communication_number", "SIOC flag":"sioc_flag", "Title":"name", "Theme": "top_theme", "Super Segment(Curr)": "super_segment", "RRP Lead EUR":"", "RRP Lead Range EUR": "rrp_lead_range_eur", "Price Range": "price_range"}, inplace=True)
uk = uk.merge(lookup, how="left")
uk = uk[uk["top_theme"] == "SUPER MARIO"]

c:\Users\gbjaudom\OneDrive - LEGO\Documents\python_projects\miscellaneous\venv_miscellaneous\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (11,13,15,16,17,18,20,22,24,38,48,50,52,53,54,55,56,58,60,62,65,66,67,69) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [7]:
def country_to_lsm(file_path):
    """
    """
    uk = pd.read_csv(file_path)
    uk = uk[['ASIN', 'Product Title', 'Ordered Revenue', 'Ordered Units', 'Average Sales Price', 'Glance Views', 'Conversion Rate', 'Country', 'Date', 'ID']]
    lookup = pd.read_excel(r"C:\Users\gbjaudom\LEGO\WE eComm CoE Analytics Team - Documents\Data\Reference docs\ASIN Lookup full.xlsx", sheet_name="Sheet1")
    lookup.rename(columns={'Material':'material', "Comms":"communication_number", "SIOC flag":"sioc_flag", "Title":"name", "Theme": "top_theme", "Super Segment(Curr)": "super_segment", "RRP Lead EUR":"", "RRP Lead Range EUR": "rrp_lead_range_eur", "Price Range": "price_range"}, inplace=True)
    uk = uk.merge(lookup, how="left")
    uk = uk[uk["top_theme"] == "SUPER MARIO"]
    return uk

In [9]:
uk = country_to_lsm(r"C:\Users\gbjaudom\LEGO\WE eComm CoE Analytics Team - Documents\Data\daily_vc_data\uk_2021_master_aba_vc_data.csv")
de = country_to_lsm(r"C:\Users\gbjaudom\LEGO\WE eComm CoE Analytics Team - Documents\Data\daily_vc_data\de_2021_master_aba_vc_data.csv")
fr = country_to_lsm(r"C:\Users\gbjaudom\LEGO\WE eComm CoE Analytics Team - Documents\Data\daily_vc_data\fr_2021_master_aba_vc_data.csv")
it = country_to_lsm(r"C:\Users\gbjaudom\LEGO\WE eComm CoE Analytics Team - Documents\Data\daily_vc_data\it_2021_master_aba_vc_data.csv")
nl = country_to_lsm(r"C:\Users\gbjaudom\LEGO\WE eComm CoE Analytics Team - Documents\Data\daily_vc_data\nl_2021_master_aba_vc_data.csv")
es = country_to_lsm(r"C:\Users\gbjaudom\LEGO\WE eComm CoE Analytics Team - Documents\Data\daily_vc_data\es_2021_master_aba_vc_data.csv")
se = country_to_lsm(r"C:\Users\gbjaudom\LEGO\WE eComm CoE Analytics Team - Documents\Data\daily_vc_data\se_2021_master_aba_vc_data.csv")

ipykernel_launcher:1: DtypeWarning: Columns (11,13,15,16,17,18,20,22,24,38,48,50,52,53,54,55,56,58,60,62,65,66,67,69) have mixed types.Specify dtype option on import or set low_memory=False.
ipykernel_launcher:2: DtypeWarning: Columns (11,13,15,16,17,18,20,22,24,38,48,50,52,53,54,55,56,58,60,62,65,66,67,69) have mixed types.Specify dtype option on import or set low_memory=False.
ipykernel_launcher:3: DtypeWarning: Columns (11,13,15,16,17,18,20,22,24,38,48,50,52,53,54,55,56,58,60,62,65,66,67,69) have mixed types.Specify dtype option on import or set low_memory=False.
ipykernel_launcher:4: DtypeWarning: Columns (11,13,15,16,17,18,20,22,24,38,48,50,52,53,54,55,56,58,60,62,65,66,67,69) have mixed types.Specify dtype option on import or set low_memory=False.
ipykernel_launcher:5: DtypeWarning: Columns (11,13,15,16,17,18,20,22,24,37,38,48,50,52,53,54,55,56,57,58,60,62,65,66,67,69) have mixed types.Specify dtype option on import or set low_memory=False.
ipykernel_launcher:6: DtypeWarning: Col

In [12]:
data_2021 = pd.concat([uk, de, es, fr, it, nl, se]).reset_index(drop=True)
data_2021

,ASIN,Product Title,Ordered Revenue,Ordered Units,Average Sales Price,Glance Views,Conversion Rate,Country,Date,ID,material,communication_number,sioc_flag,name,top_theme,super_segment,,rrp_lead_range_eur,price_range
0,B082WDX6F8,LEGO 71371 Super Mario Propeller Power-Up Pack...,241.07314,28.0,NaN,158.0,0.177215,UK,01/01/2021,B082WDX6F8_UK_01/01/2021,6288931.0,71371.0,0.0,Propeller Mario Powe,SUPER MARIO,Boys,9.99,<10 EUR,LPP
1,B082WDSG3W,LEGO 71365 Super Mario Piranha Plant Power Sli...,317.19303,14.0,NaN,218.0,0.06422,UK,01/01/2021,B082WDSG3W_UK_01/01/2021,6288919.0,71365.0,0.0,Piranha Plant Power,SUPER MARIO,Boys,29.99,<50 EUR,LPP
2,B082WCVTVR,LEGO 71370 Super Mario Fire Power-Up Pack Expa...,258.29265,30.0,NaN,191.0,0.157068,UK,01/01/2021,B082WCVTVR_UK_01/01/2021,6288929.0,71370.0,0.0,Fire Mario Power-Up,SUPER MARIO,Boys,9.99,<10 EUR,LPP
3,B082WDYKC2,LEGO 71373 Super Mario Builder Power-Up Pack E...,154.35486,18.0,NaN,102.0,0.176471,UK,01/01/2021,B082WDYKC2_UK_01/01/2021,6288935.0,71373.0,0.0,Builder Mario Power-,SUPER MARIO,Boys,9.99,<10 EUR,LPP
4,B082WDBGMP,LEGO 71366 Super Mario Boomer Bill Barrage Exp...,547.52984,26.0,NaN,227.0,0.114537,UK,01/01/2021,B082WDBGMP_UK_01/01/2021,6288921.0,71366.0,0.0,Boomer Bill Barrage,SUPER MARIO,Boys,29.99,<50 EUR,LPP
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49263,B082WF8MVV,LEGO 71369 Super Mario Striden mot slottsbosse...,656.80000,1.0,NaN,28.0,0.03571428571428571,SE,07/11/2021,B082WF8MVV_SE_07/11/2021,6288927.0,71369.0,0.0,Bowser's Castle Boss,SUPER MARIO,Boys,99.99,<150 EUR,HPP
49264,B082WF7Z4Q,LEGO 71362 Super Mario Bevakad fästning - Expa...,630.40000,2.0,NaN,24.0,0.08333333333333331,SE,07/11/2021,B082WF7Z4Q_SE_07/11/2021,6288913.0,71362.0,0.0,Guarded Fortress Exp,SUPER MARIO,Boys,49.99,<50 EUR,LPP
49265,B08WWYRGD6,LEGO 71389 Super Mario Lakitus molnvärld – för...,499.35000,2.0,NaN,14.0,0.14285714285714285,SE,07/11/2021,B08WWYRGD6_SE_07/11/2021,6332719.0,71389.0,0.0,Lakitu Sky World Exp,SUPER MARIO,Boys,39.99,<50 EUR,LPP
49266,B08WWQ43C8,LEGO 71392 Super Mario Frog Mario Power-Up Pac...,0.00000,0.0,NaN,0.0,—,SE,07/11/2021,B08WWQ43C8_SE_07/11/2021,6332727.0,71392.0,0.0,Frog Mario Power-Up,SUPER MARIO,Boys,9.99,<10 EUR,LPP


In [15]:
data_2020 = pd.read_csv(r"C:\Users\gbjaudom\Onedrive - LEGO\Documents\python_projects\miscellaneous\data\processed\2020_aug_dec_daily_gv_cvr_or.csv", index_col=0)
lookup = pd.read_excel(r"C:\Users\gbjaudom\LEGO\WE eComm CoE Analytics Team - Documents\Data\Reference docs\ASIN Lookup full.xlsx", sheet_name="Sheet1")
lookup.rename(columns={'Material':'material', "Comms":"communication_number", "SIOC flag":"sioc_flag", "Title":"name", "Theme": "top_theme", "Super Segment(Curr)": "super_segment", "RRP Lead EUR":"", "RRP Lead Range EUR": "rrp_lead_range_eur", "Price Range": "price_range"}, inplace=True)
data_2020 = data_2020.merge(lookup, how="left")
data_2020 = data_2020[data_2020["top_theme"] == "SUPER MARIO"]


In [18]:
final_df = pd.concat([data_2020, data_2021]).reset_index(drop=True)

In [19]:
final_df.to_csv(r"C:\Users\gbjaudom\Onedrive - LEGO\Documents\python_projects\miscellaneous\data\processed\2020_aug_2021_nov_gv_cvr_or.csv")

## Daily to Weekly/Monthly

In [3]:
import pandas as pd

final_df = pd.read_csv(r"C:\Users\gbjaudom\Onedrive - LEGO\Documents\python_projects\miscellaneous\data\processed\2020_aug_2021_nov_gv_cvr_or.csv", index_col=0)

In [6]:
final_df = final_df[final_df["Country"] != "SE"]


In [7]:
final_df["Date"] = pd.to_datetime(final_df["Date"], dayfirst=True)
final_df["week"] = final_df["Date"].dt.week
final_df["month"] = final_df["Date"].dt.month
final_df["year"] = final_df["Date"].dt.year
final_df.head()

ipykernel_launcher:2: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.


,ASIN,Product Title,Ordered Revenue,Ordered Units,Average Sales Price,Glance Views,Conversion Rate,Country,Date,ID,...,sioc_flag,name,top_theme,super_segment,Unnamed: 17,rrp_lead_range_eur,price_range,week,month,year
0,B082WDQHZQ,LEGO 71360 Super Mario Abenteuer mit Mario – S...,37998.37,938,40.509989,29410.0,0.031894,DE,2020-08-01,B082WDQHZQ_DE_01/08/2020,...,0,Adventures with Mari,SUPER MARIO,Boys,59.99,<80 EUR,MPP,31,8,2020
1,B082WF8MVV,LEGO 71369 Super Mario Bowsers Festung – Erwei...,9123.23,109,83.699358,6049.0,0.018020,DE,2020-08-01,B082WF8MVV_DE_01/08/2020,...,0,Bowser's Castle Boss,SUPER MARIO,Boys,99.99,<150 EUR,HPP,31,8,2020
2,B082WF8MB5,LEGO 71367 Super Mario Marios Haus und Yoshi –...,5348.33,216,24.760787,4500.0,0.048000,DE,2020-08-01,B082WF8MB5_DE_01/08/2020,...,0,Mario's House & Yosh,SUPER MARIO,Boys,29.99,<50 EUR,LPP,31,8,2020
3,B082WF7Z4Q,LEGO 71362 Super Mario Bewachte Festung – Erwe...,3360.80,80,42.010000,5529.0,0.014469,DE,2020-08-01,B082WF7Z4Q_DE_01/08/2020,...,0,Guarded Fortress Exp,SUPER MARIO,Boys,49.99,<50 EUR,LPP,31,8,2020
4,B082WDZM4T,LEGO 71368 Super Mario Toads Schatzsuche – Erw...,2661.91,40,66.547750,2233.0,0.017913,DE,2020-08-01,B082WDZM4T_DE_01/08/2020,...,0,Toad's Treasure Hunt,SUPER MARIO,Boys,79.99,<80 EUR,MPP,31,8,2020


In [8]:
# Formatting
final_df = final_df.replace("—", 0)
final_df["ASP"] = final_df["Ordered Revenue"] / final_df["Ordered Units"]

ASIN  Product Title  Ordered Revenue  Ordered Units  \
year month                                                        
2020 8      2460           2460             2460           2460   
     9      2492           2492             2492           2492   
     10     5122           5122             5122           5122   
     11     4984           4984             4984           4984   
     12     5530           5530             5530           5530   
2021 1      3875           3239             3875           3875   
     2      3436           2852             3436           3436   
     3      3631           2994             3631           3631   
     4      3511           2914             3511           3511   
     5      3744           3098             3744           3744   
     6      3308           2720             3308           3308   
     7      3860           3229             3860           3860   
     8      5036           5036             5036           5036   
     9      4989           4989             4989           4989   
     10     5218           5218             5218           5218   
     11      509            509              509            509   

            Average Sales Price  Glance Views  Conversion Rate  Country  Date  \
year month                                                                      
2020 8                     2278          2460             2344     2460  2460   
     9                     2273          2492             2344     2492  2492   
     10                    4705          5122             4769     5122  5122   
     11                    4410          4984             4360     4984  4984   
     12                    4314          5508             4390     5530  5530   
2021 1                        0          3875             3465     3875  3875   
     2                        0          3436             3141     3436  3436   
     3                        0          3631             3196     3631  3631   
     4                        0          3511             2930     3511  3511   
     5                        0          3744             2953     3744  3744   
     6                        0          3308             2695     3308  3308   
     7                        0          3860             3184     3860  3860   
     8                     2021          5035             3953     5036  5036   
     9                        0          4989             3747     4989  4989   
     10                       0          5218             3922     5218  5218   
     11                       0           509              383      509   509   

              ID  ...  communication_number  sioc_flag  name  top_theme  \
year month        ...                                                     
2020 8      2460  ...                  2460       2460  2460       2460   
     9      2492  ...                  2492       2492  2492       2492   
     10     5122  ...                  5122       5122  5122       5122   
     11     4984  ...                  4984       4984  4984       4984   
     12     5530  ...                  5530       5530  5530       5530   
2021 1      3875  ...                  3875       3875  3875       3875   
     2      3436  ...                  3436       3436  3436       3436   
     3      3631  ...                  3631       3631  3631       3631   
     4      3511  ...                  3511       3511  3511       3511   
     5      3744  ...                  3744       3744  3744       3744   
     6      3308  ...                  3308       3308  3308       3308   
     7      3860  ...                  3860       3860  3860       3860   
     8      5036  ...                  5036       5036  5036       5036   
     9      4989  ...                  4989       4989  4989       4989   
     10     5218  ...                  5218       5218  5218       5218   
     11      509  ...                   509        509   509        509

In [80]:
grp_weekly_df = final_df.groupby(["year", "month", "week", "ASIN", "Product Title"]).agg({'Ordered Revenue':'sum', 'Ordered Units':'sum', 'ASP':'mean', 'Glance Views':'mean', "Conversion Rate": "mean"}).reset_index()
grp_weekly_df

,year,month,week,ASIN,Product Title,Ordered Revenue,Ordered Units,ASP,Glance Views,Conversion Rate
0,2020,8,31,B082WCVTVR,"LEGO 71370 Super Mario Feuer-Mario - Anzug, Er...",1234.80000,147,8.400000,761.500000,0.093278
1,2020,8,31,B082WCVTVR,LEGO 71370 Super Mario Costume Mario de feu,216.32000,26,8.320000,195.500000,0.065980
2,2020,8,31,B082WCVTVR,"LEGO Super Mario Fuoco - Power Up Pack, Espans...",171.99000,21,8.190000,226.500000,0.049299
3,2020,8,31,B082WCVTVR,LEGO 71370 Super Mario Fire Power-Up Pack Expa...,1219.73152,140,8.712368,551.500000,0.124886
4,2020,8,31,B082WCVTVR,LEGO 71370 Super Mario Pack Potenciador: Mario...,149.06000,13,10.213750,206.000000,0.031226
...,...,...,...,...,...,...,...,...,...,...
9274,2021,11,44,B0984RGDVH,LEGO 71391 Super Mario Bowsers Luftschiff – Er...,7148.98000,113,63.306087,720.000000,0.054046
9275,2021,11,44,B0984RGDVH,LEGO 71391 Super Mario Ensemble d’Extension La...,3599.52000,48,74.990000,279.000000,0.057966
9276,2021,11,44,B0984RGDVH,LEGO 71391 Super Mario Set de Expansión: Forta...,159.90000,2,79.950000,18.666667,0.034101
9277,2021,11,44,B0984RGDVH,LEGO Super Mario Il Veliero Volante di Bowser ...,510.58000,7,72.940000,117.666667,0.021328


In [87]:
grp_monthly_df = final_df.groupby(["year", "month"]).agg({'Ordered Revenue':'sum', 'Ordered Units':'sum', 'ASP':'mean', 'Glance Views':'mean', "Conversion Rate": "mean"}).reset_index()
grp_monthly_df["ASP"] = grp_monthly_df["Ordered Revenue"] / grp_monthly_df["Ordered Units"]
grp_monthly_df["year_month"] = grp_monthly_df["year"].astype(str) + "_" + grp_monthly_df["month"].astype(str)
grp_monthly_df

,year,month,Ordered Revenue,Ordered Units,ASP,Glance Views,Conversion Rate,year_month
0,2020,8,2.560894e+06,83847,30.542464,620.632114,0.061471,2020_8
1,2020,9,1.905269e+06,65348,29.155742,417.778892,0.093087,2020_9
2,2020,10,3.770091e+06,138425,27.235620,392.895158,0.091554,2020_10
3,2020,11,6.438596e+06,249196,25.837479,633.742376,0.084144,2020_11
4,2020,12,5.185042e+06,226692,22.872629,463.219317,0.090089,2020_12
5,2021,1,1.343354e+06,69689,19.276409,231.235355,0.089914,2021_1
6,2021,2,1.210718e+06,66106,18.314791,207.261932,0.098764,2021_2
7,2021,3,1.275999e+06,74980,17.017856,196.881851,0.109245,2021_3
8,2021,4,7.742906e+05,49887,15.520890,131.697807,0.104582,2021_4
9,2021,5,7.589265e+05,40132,18.910757,123.681357,0.099016,2021_5


In [83]:
import plotly.graph_objects as go

fig = go.Figure()
fig.add_trace(go.Bar(
    x=grp_monthly_df["year_month"], y=grp_monthly_df["ASP"],
))
fig.show()

In [84]:
fig = go.Figure()
fig.add_trace(go.Bar(
    x=grp_monthly_df["year_month"], y=grp_monthly_df["Glance Views"],
))
fig.show()

In [85]:
fig = go.Figure()
fig.add_trace(go.Bar(
    x=grp_monthly_df["year_month"], y=grp_monthly_df["Conversion Rate"],
))
fig.show()

In [19]:
# ASINs by Month/Market
final_df["year_month"] = final_df["year"].astype(str) + "_" + final_df["month"].astype(str)

for time in final_df["year_month"].unique():
    df = final_df[final_df["year_month"] == time]
    print(time, len(df["ASIN"].unique()))

2020_8 14
2020_9 14
2020_10 14
2020_11 14
2020_12 21
2021_1 21
2021_2 21
2021_3 21
2021_4 22
2021_5 22
2021_6 23
2021_7 23
2021_8 29
2021_9 30
2021_10 30
2021_11 30


In [34]:
de = final_df[final_df["Country"] == "NL"]
grp_monthly_df = de.groupby(["year", "month"]).agg({'Ordered Revenue':'sum', 'Ordered Units':'sum', 'ASP':'mean', 'Glance Views':'sum', "Conversion Rate": "mean"}).reset_index()
grp_monthly_df["ASP"] = grp_monthly_df["Ordered Revenue"] / grp_monthly_df["Ordered Units"]
grp_monthly_df["year_month"] = grp_monthly_df["year"].astype(str) + "_" + grp_monthly_df["month"].astype(str)
grp_monthly_df

,year,month,Ordered Revenue,Ordered Units,ASP,Glance Views,Conversion Rate,year_month
0,2020,8,37535.10,1137,33.012401,17166.0,0.087317,2020_8
1,2020,9,35881.02,1425,25.179663,22487.0,0.094885,2020_9
2,2020,10,79111.74,4254,18.597024,43568.0,0.123918,2020_10
3,2020,11,276832.88,17938,15.432762,152830.0,0.101580,2020_11
4,2020,12,98435.40,5398,18.235532,64442.0,0.078886,2020_12
5,2021,1,52324.17,2730,19.166363,29800.0,0.100458,2021_1
6,2021,2,30683.43,2016,15.219955,18508.0,0.135413,2021_2
7,2021,3,41442.02,2553,16.232675,24447.0,0.098992,2021_3
8,2021,4,15782.91,1167,13.524344,16062.0,0.080444,2021_4
9,2021,5,21223.41,1302,16.300622,17784.0,0.085646,2021_5


In [28]:
de[de["year_month"] == "2020_8"]

,ASIN,Product Title,Ordered Revenue,Ordered Units,Average Sales Price,Glance Views,Conversion Rate,Country,Date,ID,...,top_theme,super_segment,Unnamed: 17,rrp_lead_range_eur,price_range,week,month,year,ASP,year_month
0,B082WDQHZQ,LEGO 71360 Super Mario Abenteuer mit Mario – S...,37998.37,938,40.509989,29410.0,0.031894,DE,2020-08-01,B082WDQHZQ_DE_01/08/2020,...,SUPER MARIO,Boys,59.99,<80 EUR,MPP,31,8,2020,40.509989,2020_8
1,B082WF8MVV,LEGO 71369 Super Mario Bowsers Festung – Erwei...,9123.23,109,83.699358,6049.0,0.018020,DE,2020-08-01,B082WF8MVV_DE_01/08/2020,...,SUPER MARIO,Boys,99.99,<150 EUR,HPP,31,8,2020,83.699358,2020_8
2,B082WF8MB5,LEGO 71367 Super Mario Marios Haus und Yoshi –...,5348.33,216,24.760787,4500.0,0.048000,DE,2020-08-01,B082WF8MB5_DE_01/08/2020,...,SUPER MARIO,Boys,29.99,<50 EUR,LPP,31,8,2020,24.760787,2020_8
3,B082WF7Z4Q,LEGO 71362 Super Mario Bewachte Festung – Erwe...,3360.80,80,42.010000,5529.0,0.014469,DE,2020-08-01,B082WF7Z4Q_DE_01/08/2020,...,SUPER MARIO,Boys,49.99,<50 EUR,LPP,31,8,2020,42.010000,2020_8
4,B082WDZM4T,LEGO 71368 Super Mario Toads Schatzsuche – Erw...,2661.91,40,66.547750,2233.0,0.017913,DE,2020-08-01,B082WDZM4T_DE_01/08/2020,...,SUPER MARIO,Boys,79.99,<80 EUR,MPP,31,8,2020,66.547750,2020_8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2386,B082WCVTVR,"LEGO 71370 Super Mario Feuer-Mario - Anzug, Er...",509.32,68,7.490000,505.0,0.134653,DE,2020-08-31,B082WCVTVR_DE_31/08/2020,...,SUPER MARIO,Boys,9.99,<10 EUR,LPP,36,8,2020,7.490000,2020_8
2387,B082WDX6F8,LEGO 71371 Super Mario Propeller-Mario - Anzug...,477.40,62,7.700000,536.0,0.115672,DE,2020-08-31,B082WDX6F8_DE_31/08/2020,...,SUPER MARIO,Boys,9.99,<10 EUR,LPP,36,8,2020,7.700000,2020_8
2388,B082WD5BV6,"LEGO 71372 Super Mario Katzen-Mario - Anzug, E...",458.85,61,7.522131,430.0,0.141860,DE,2020-08-31,B082WD5BV6_DE_31/08/2020,...,SUPER MARIO,Boys,9.99,<10 EUR,LPP,36,8,2020,7.522131,2020_8
2389,B082WDYKC2,LEGO 71373 Super Mario Baumeister-Mario - Anzu...,258.85,31,8.350000,330.0,0.093939,DE,2020-08-31,B082WDYKC2_DE_31/08/2020,...,SUPER MARIO,Boys,9.99,<10 EUR,LPP,36,8,2020,8.350000,2020_8


In [23]:
de.groupby(["year", "month"]).agg({'Ordered Revenue':'sum', 'Ordered Units':'sum', 'ASP':'mean', 'Glance Views':'mean', "Conversion Rate": "mean"}).reset_index()

,year,month,Ordered Revenue,Ordered Units,ASP,Glance Views,Conversion Rate
0,2020,8,1112415.67,36647,25.449184,1563.060325,0.067282
1,2020,9,692803.48,26889,22.277647,899.918854,0.133793
2,2020,10,1089063.90,42590,NaN,680.658986,0.124954
3,2020,11,1752885.56,61492,NaN,856.019048,0.092070
4,2020,12,1172306.48,44386,NaN,508.781385,0.105466
5,2021,1,414511.68,22254,NaN,394.895223,0.106154
6,2021,2,349633.54,21113,NaN,355.686644,0.118491
7,2021,3,472137.74,28662,19.411857,434.787167,0.119723
8,2021,4,160782.83,9311,19.538748,181.755337,0.095831
9,2021,5,148040.46,8359,25.724835,160.676145,0.096202
